In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

In [12]:
pickle_file = '/Users/RickardMaus/Documents/notMNIST/notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


In [13]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 0 to [1.0, 0.0, 0.0 ...], 1 to [0.0, 1.0, 0.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [14]:
# With gradient descent training, even this much data is prohibitive.
# Subset the training data for faster turnaround.
train_subset = 10000

graph = tf.Graph()
with graph.as_default():

  # Input data.
  # Load the training, validation and test data into constants that are
  # attached to the graph.
  tf_train_dataset = tf.constant(train_dataset[:train_subset, :])
  tf_train_labels = tf.constant(train_labels[:train_subset])
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  # These are the parameters that we are going to be training. The weight
  # matrix will be initialized using random values following a (truncated)
  # normal distribution. The biases get initialized to zero.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  # We multiply the inputs with the weight matrix, and add biases. We compute
  # the softmax and cross-entropy (it's one operation in TensorFlow, because
  # it's very common, and it can be optimized). We take the average of this
  # cross-entropy across all training examples: that's our loss.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
  
  # Optimizer.
  # We are going to find the minimum of this loss using gradient descent.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  # These are not part of training, but merely here so that we can report
  # accuracy figures as we train.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [5]:
num_steps = 801

def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

with tf.Session(graph=graph) as session:
  # This is a one-time operation which ensures the parameters get initialized as
  # we described in the graph: random weights for the matrix, zeros for the
  # biases. 
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    # Run the computations. We tell .run() that we want to run the optimizer,
    # and get the loss value and the training predictions returned as numpy
    # arrays.
    _, l, predictions = session.run([optimizer, loss, train_prediction])
    if (step % 100 == 0):
      print('Loss at step %d: %f' % (step, l))
      print('Training accuracy: %.1f%%' % accuracy(
        predictions, train_labels[:train_subset, :]))
      # Calling .eval() on valid_prediction is basically like calling run(), but
      # just to get that one numpy array. Note that it recomputes all its graph
      # dependencies.
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Loss at step 0: 16.985409
Training accuracy: 10.2%
Validation accuracy: 13.5%
Loss at step 100: 2.277820
Training accuracy: 72.0%
Validation accuracy: 70.7%
Loss at step 200: 1.849468
Training accuracy: 75.0%
Validation accuracy: 72.8%
Loss at step 300: 1.612199
Training accuracy: 76.1%
Validation accuracy: 73.8%
Loss at step 400: 1.448978
Training accuracy: 77.1%
Validation accuracy: 74.3%
Loss at step 500: 1.326725
Training accuracy: 77.7%
Validation accuracy: 74.6%
Loss at step 600: 1.230201
Training accuracy: 78.3%
Validation accuracy: 74.7%
Loss at step 700: 1.151224
Training accuracy: 78.9%
Validation accuracy: 74.9%
Loss at step 800: 1.085018
Training accuracy: 79.3%
Validation accuracy: 75.0%
Test accuracy: 82.8%


In [6]:
batch_size = 128

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [7]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 22.244795
Minibatch accuracy: 5.5%
Validation accuracy: 8.8%
Minibatch loss at step 500: 1.788540
Minibatch accuracy: 75.8%
Validation accuracy: 75.0%
Minibatch loss at step 1000: 1.316192
Minibatch accuracy: 74.2%
Validation accuracy: 76.2%
Minibatch loss at step 1500: 1.234050
Minibatch accuracy: 75.8%
Validation accuracy: 76.7%
Minibatch loss at step 2000: 1.483200
Minibatch accuracy: 75.0%
Validation accuracy: 77.6%
Minibatch loss at step 2500: 1.180397
Minibatch accuracy: 72.7%
Validation accuracy: 77.8%
Minibatch loss at step 3000: 0.546088
Minibatch accuracy: 86.7%
Validation accuracy: 78.6%
Test accuracy: 86.3%


In [45]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

# Parameters
learning_rate = 0.0001
training_epochs = 2000
batch_s = 100
display_step = 1

# Network Parameters
n_hidden = 1024
n_input = 784
n_classes = 10

graph = tf.Graph()
with graph.as_default():
    # tf Graph input
    tf_train_data = tf.placeholder(tf.float32, shape=(batch_s, n_input))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_s, n_classes))
    tf_valid_data = tf.constant(valid_dataset)
    tf_test_data = tf.constant(test_dataset)

    # Store layers weight & bias
    weights = {
        'L1' : tf.Variable(tf.random_normal([n_input, n_hidden])),
        'H1' : tf.Variable(tf.random_normal([n_hidden, n_hidden])),
        'out' : tf.Variable(tf.random_normal([n_hidden, n_classes]))
    }
    biases = {
        'b1' : tf.Variable(tf.random_normal([n_hidden])),
        'b2' : tf.Variable(tf.random_normal([n_hidden])),
        'out' : tf.Variable(tf.random_normal([n_classes]))
    }


    # Create model
    def classifier(x):
        # Hidden fully connected layer with 1024 neurons
        layer_1 = tf.add(tf.matmul(x, weights['L1']), biases['b1'])
        # Hidden fully connected layer with 1024 neurons
        layer_2 = tf.add(tf.matmul(tf.nn.relu(layer_1), weights['H1']), biases['b2'])
        # Output fully connected layer with a neuron for each class
        out_layer = tf.matmul(layer_2, weights['out']) + biases['out']
        return out_layer

    # Construct model
    logits = classifier(tf_train_data)
    logits_v = classifier(tf_valid_data)
    logits_t = classifier(tf_test_data)
    
    # Define loss and optimizer
    loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
        logits=logits, labels=tf_train_labels))
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    train_op = optimizer.minimize(loss_op)
    # Initializing the variables

    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(logits_v)
    test_prediction = tf.nn.softmax(tf_test_data)

with tf.Session(graph=graph) as sess:
    tf.global_variables_initializer().run()
    # Training cycle
    for epoch in range(training_epochs):
        avg_cost = 0.
        offset = (epoch * batch_s)
        batch_data = train_dataset[offset:(offset+batch_s), :]
        batch_labels = train_labels[offset:(offset+batch_s), :]
        feed_dict = {tf_train_data : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = sess.run(
            [train_op, loss_op, train_prediction], feed_dict=feed_dict)
        if (epoch % 200 == 0):
            print("Minibatch loss at step %d: %f" % (epoch, l))
            print("Minibatch accuracy: %.1f%%" %  accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
    print("Test accuracy: %1f.%%" % accuracy(test_prediction.eval(), test_labels))



Minibatch loss at step 0: 18667.630859
Minibatch accuracy: 9.0%
Validation accuracy: 8.8%
Minibatch loss at step 200: 1778.370605
Minibatch accuracy: 70.0%
Validation accuracy: 66.6%
Minibatch loss at step 400: 2007.584961
Minibatch accuracy: 68.0%
Validation accuracy: 70.8%
Minibatch loss at step 600: 1958.964355
Minibatch accuracy: 69.0%
Validation accuracy: 72.4%
Minibatch loss at step 800: 1639.897827
Minibatch accuracy: 74.0%
Validation accuracy: 73.8%
Minibatch loss at step 1000: 1627.600342
Minibatch accuracy: 70.0%
Validation accuracy: 74.3%
Minibatch loss at step 1200: 1150.001465
Minibatch accuracy: 77.0%
Validation accuracy: 75.4%
Minibatch loss at step 1400: 1667.611816
Minibatch accuracy: 70.0%
Validation accuracy: 75.8%
Minibatch loss at step 1600: 1068.443726
Minibatch accuracy: 76.0%
Validation accuracy: 76.1%
Minibatch loss at step 1800: 851.528748
Minibatch accuracy: 85.0%
Validation accuracy: 76.2%
Test accuracy: 3.350000.%


In [57]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

# Parameters
learning_rate = 0.1
training_epochs = 1000
batch_s = 200
display_step = 1

# Network Parameters
n_hidden = 1024
n_input = 784
n_classes = 10

graph = tf.Graph()
with graph.as_default():
    # tf Graph input
    tf_train_data = tf.placeholder(tf.float32, shape=(batch_s, n_input))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_s, n_classes))
    tf_valid_data = tf.constant(valid_dataset)
    tf_test_data = tf.constant(test_dataset)

    # Store layers weight & bias
    weights = {
        'L1' : tf.Variable(tf.random_normal([n_input, n_hidden])),
        'out' : tf.Variable(tf.random_normal([n_hidden, n_classes]))
    }
    biases = {
        'b1' : tf.Variable(tf.random_normal([n_hidden])),
        'out' : tf.Variable(tf.random_normal([n_classes]))
    }


    # Create model
    def classifier(x):
        # Hidden fully connected layer with 1024 neurons
        layer_1 = tf.add(tf.matmul(x, weights['L1']), biases['b1'])
        # Output fully connected layer with a neuron for each class
        out_layer = tf.matmul(tf.nn.relu(layer_1), weights['out']) + biases['out']
        return out_layer

    # Construct model
    logits = classifier(tf_train_data)
    logits_v = classifier(tf_valid_data)
    logits_t = classifier(tf_test_data)
    
    # Define loss and optimizer
    loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
        logits=logits, labels=tf_train_labels))
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    train_op = optimizer.minimize(loss_op)
    # Initializing the variables

    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(logits_v)
    test_prediction = tf.nn.softmax(logits_t)

with tf.Session(graph=graph) as sess:
    tf.global_variables_initializer().run()
    # Training cycle
    for epoch in range(training_epochs):
        avg_cost = 0.
        offset = (epoch * batch_s)
        batch_data = train_dataset[offset:(offset+batch_s), :]
        batch_labels = train_labels[offset:(offset+batch_s), :]
        feed_dict = {tf_train_data : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = sess.run(
            [train_op, loss_op, train_prediction], feed_dict=feed_dict)
        if (epoch % 200 == 0):
            print("Minibatch loss at step %d: %f" % (epoch, l))
            print("Minibatch accuracy: %.1f%%" %  accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
    print("Test accuracy: %1f.%%" % accuracy(test_prediction.eval(), test_labels))



Minibatch loss at step 0: 406.713287
Minibatch accuracy: 9.5%
Validation accuracy: 28.1%
Minibatch loss at step 200: 28.154730
Minibatch accuracy: 82.0%
Validation accuracy: 78.4%
Minibatch loss at step 400: 34.548660
Minibatch accuracy: 77.0%
Validation accuracy: 80.2%
Minibatch loss at step 600: 21.428101
Minibatch accuracy: 81.5%
Validation accuracy: 81.3%
Minibatch loss at step 800: 16.862316
Minibatch accuracy: 81.5%
Validation accuracy: 81.8%
Test accuracy: 89.500000.%
